# **Data Collectors and Landing Zone**

### *Required packages:*

In [25]:
# !pip install opendatasets
# !pip install kaggle
# !pip install openpyxl
# !pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 5.8 MB/s eta 0:00:00m eta 0:00:010:00:01


In [26]:
import numpy as np
import pandas as pd
import requests
import os
import opendatasets as od
import csv
import openpyxl
import pyarrow

### Kaggle Configuration - Drive

(Requereix del fitxer `kaggle_fitxer.json`)

In [11]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/home/oscarmolina/.kaggle’: File exists


In [12]:
!cp ./kaggle_user_config.json ~/.kaggle/kaggle.json

In [13]:
!cd ~/.kaggle && ls

kaggle.json


### CODE

#### Downloading datasets

##### UNECE

In [14]:
# URL del dataset UNECE
unece_db = pd.read_json('https://query.data.world/s/qd6kjqguuvb573mffbpfysgpudi2pv?dws=00000')
unece_db.to_csv('unece_db.csv', index=False)
!mv ./unece_db.csv ../data/unece_db.csv

##### Kaggle - Mental Health

In [17]:
# API Kaggle Mental Health
!kaggle datasets download  -d 'thedevastator/uncover-global-trends-in-mental-health-disorder'

 81%|██████████████████████████████▌       | 1.00M/1.24M [00:00<00:00, 1.50MB/s]
100%|██████████████████████████████████████| 1.24M/1.24M [00:00<00:00, 1.80MB/s]


In [19]:
#!sudo apt-get install unzip
!unzip uncover-global-trends-in-mental-health-disorder.zip -d ../data/
!rm uncover-global-trends-in-mental-health-disorder.zip

Archive:  uncover-global-trends-in-mental-health-disorder.zip
  inflating: ../data/Mental health Depression disorder Data.csv  


##### USA Census

In [20]:
# API United States Census
variables = "NAME","YR","AREA_KM2","CBR","CDR","DEATHS","E0","GRR","IMR","MEDAGE","MR0_4","POP_DENS","genc standard countries and areas"
url_base = "https://api.census.gov/data/timeseries/idb/5year?get=NAME,YR,AREA_KM2,CBR,CDR,DEATHS,E0,GRR,IMR,MEDAGE,MR0_4,POP_DENS&for=genc%20standard%20countries%20and%20areas:*".format(variables)

# Llista per guardar les dades
data = []

# Fem una sol·licitud a l'API per agafar dades des de l'any 1990 fins al 2019
for year in range(1990, 2019):
    url = "{}&time={}".format(url_base, year)
    response = requests.get(url)
    if response.status_code == 200:
        data.extend(response.json()[1:])

# Guardar los datos en un archivo CSV
csv_filename = "dades_demogr_census.csv"
with open(csv_filename, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["NAME","YR","AREA_KM2","CBR","CDR","DEATHS","E0","GRR","IMR","MEDAGE","MR0_4","POP_DENS","genc standard countries and areas", "code"])
    csv_writer.writerows(data)

print("Les dades s'han guardat a", csv_filename)

!mv ./dades_demogr_census.csv ../data/dades_demogr_census.csv


Les dades s'han guardat a dades_demogr_census.csv


#### Rename

In [22]:
!mv ../data/dades_demogr_census.csv ../data/demography.csv
!mv ../data/unece_db.csv ../data/society.csv
#!mv ../data/Mental health Depression disorder Data.csv ../data/mental_health.csv

#### Storing files in a suitable format (Parquet)


In [27]:
# Unece_db
unece_db = pd.read_csv('../data/society.csv')
unece_db.to_parquet('../data/society.parquet')

# Kaggle db
kaggle_db = pd.read_csv('../data/mental_health.csv', decimal=',')
kaggle_db.to_parquet('../data/mental_health.parquet')

# United State Census db
us_census_db = pd.read_csv('../data/demography.csv')
us_census_db.to_parquet('../data/demography.parquet')

/tmp/ipykernel_68850/3970328731.py:6: DtypeWarning: Columns (7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  kaggle_db = pd.read_csv('../data/mental_health.csv', decimal=',')
